> 1. DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.
> 2. SET THE "General Access" OF THE COPIED NOTEBOOK TO "Anyone with the link" BY CLICKING ON "Share" TO ENABLE SHARING WITH YOUR PEERS FOR REVIEW.


### This project is from the *Causal Inference for Data Science course on CoRise.* Learn more about the course [here](https://corise.com/course/causal-inference-for-data-science).



---





# Week 2 Project: Applying Instrumental Variables and Regression Discontinuity
***

Welcome to the second project for Causal Inference for Data Science!


## Scenario

It is your first day after the big executive meeting at Tongass where you presented the results of your linear regression analysis. It was well received. The CEO even gave you a fist bump! (Per our experience this can actually happen if your CEO is cool.)

Now the VP of In Store Sales wants to use your results to push for expanding the number of stores, but the VP of Onlines Sales is pushing back, saying the results are just correlational! (This VP took a stats class in their MBA program.)

The VP of In Store Sales comes to your desk and asks: "Isn't there some data science voodoo we can do to get better results and prove in store sales boost customer value?"

You reply "Definitely! I am a causal inference wizard. But getting a straight answer will require me to run some experiments and possibly offer discounts to change customer behavior."

"My entire team is at your disposal. If we can show this, I will have the evidence I need to double store openings and turbo charge our growth.", the VP of In Store Sales replies.

You say, "Wonderful. I'll get right to it!"


## Project notes

* We will work with the same data set as last week and use it to tackle instrumental variables and regression discontinuity through different experiments.

* As before we attempt to strike a balance between providing concrete steps to follow and making room for exploration. That said, we encourage you to explore: The best way to become a causal inference expert is to attack a single problem from multiple angles to see how different modeling choices affect an analysis.

In [ ]:
#!pip install stargazer
#!pip install sklearn
#!pip install rdrobust

In [ ]:
# loading necessary packages
%matplotlib inline

import numpy as np
import pandas as pd
import datetime as dt
np.random.seed(44)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)

import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.gmm import IV2SLS
from stargazer.stargazer import Stargazer
from rdrobust import rdrobust,rdbwselect,rdplot

<br/>

## I. Load the data
***

As before the data set is in the file called `tongass_transactions.csv`.

Note: The data set is at the **transaction level**, not the customer level. Any given customer can (and likely does) have multiple transactions. Some measures and fields are at the customer level, while others are at the transaction level. It will be up to you to manipulate this data set so that it can be used for analysis. The question we're interested in is fundamentally about *customers, not transactions,* which means we'll likely need to aggregate the data to the customer level to make it useful.

Below, we define the fields that are relevant for this week:
- `customer_id`: the unique identifier for a given customer
- `age`: the age of the customer
- `income`: the income of the customer 
- `state`: the customer's state of residence
- `received_re`*: 1 if customer received an email in a prior AB test or 0 if customer received no email
- `received_in_store_re`*: 1 for treatment email in  AB test that offered a free gift in store with purchase and 0 for control email with no gift
- `distance`: the distance (in miles) from a customer's home to the nearest Tongass store
- `tx_order`: whether the transaction is the customer's first, second, third... etc.
- `amount`: the dollar value of the transaction
- `tx_date`: the date of the transaction
- `is_credit`: whether the transaction involved a credit card or a different payment method (1 if credit card, 0 if other)
- `in_store`: whether the transaction happened in a physical store (1 if yes, 0 if no and happened on tongass.com)

*Note `received_re` and `received_in_store_re` are defined at the customer level even if not all transactions by the customer occurred within the test window as specified below.

In [ ]:
# TODO: read in the data to an object called df
url = 'https://raw.githubusercontent.com/dsaber/co-rise-causal-inference/main/tongass_transactions.csv'
df = pd.read_csv(url)
df.head()

<br/>

## II. Instrumental Variables
***

<br/>

We first tackle answering our causal question of interest using instrumental variables.

After talking to the in store shopping team about past experiments, you discover there was a previous AB test that tried to boost in store visits. The team wanted to assess the impact of providing a free gift with an in store purchase on foot traffic for customers living close to a store. The test was successful in boosting in store visits.

Aha, this seems like a good candidate for a randomized encouragement trial! It is likely that the free gift nudged people to visit the store, allowing us to use it to estimate the impact of in store shopping on customer value.

### A. Checking the Assumptions

Remember, our causal question of interest is estimating the impact of in store visits (treatment T) on customer value (outcome Y). We want to find an instrument Z that we can use to estimate this causal effect.

In a causal diagram it looks like this where in store vists T causes customer value Y but both are impacted by various confounders X. Our instrument Z allows us to estimate the causal effect cleanly without worrying about confouders:

                                      Z -- T -- Y
                                            \ X /

A good IV must satisfy three properties:

*   Random assignment
*   Strong first stage
*   Exclusion restriction

A candidate for our instrument Z is the email treatment (`received_in_store_re`) sent to customers in the past AB test.

We will test the strong first stage assumption later, but take some time to ask yourself:

(1) Is this instrument we are using random of any other confounders? Why or why not?

(2) Does this instrument plausibly impact the outcome (customer value) only through its impact on the treatment (in store visits)? Why or why not?

### B. Checking the Data

Now that we are convinced the assumptions for a valid instrument are satisfied, let's investigate and prepare the data for analysis.

The email AB test that we want to use as a randomized encouragement trial ran from November, 2020 to January, 2021. On November 1 the email went out to customers, and the free gift promotion expired on February 1, 2021.

Let's start by counting the number of customers in the AB test (`received_re = 1`) and comparing them to those that were not (`received_re = 0`) on age, income, distance from a store, and credit card purchases.

How similar are they? Will this impact the generalization of our IV analysis?

In [ ]:
# TODO: compare those in the AB test to those not in the test

### C. Preparing the Data

Now let's prepare our IV dataset. There are a few choices we need to make in aggregating and subsetting down to the relevant rows.

*   Do we want to limit the data to transactions during the randomized encouragement trial / AB test?
*   Does it makes sense to include future transactions after the AB test ended to incorporate future value? Could this violate the exclusion restriction assumption?
*   What controls should we include in our data set if any?
*   How do we ensure the controls don't cause bias in our model? (Hint: controls should only be defined using data prior to the date that the treatment occurred.)
*   Is customer data prior to the test such as their prior transactions or prior in store visits valuable?
*   How should we define in store visits (treatment) and customer value (outcome) here? Do we want to use the count of in store visits, if the customer had an in store visit, or something else?

Based on what you think is most sensible, prepare a dataframe called `df_iv` with the data you will use for IV regression.

In [ ]:
# date range covered by data set
date_range = pd.date_range(start='2020-01-01', end='2022-12-31', freq='M')

# time period covered by randomized encouragement trial
randomized_encouragement_dates = [d.strftime('%Y-%m-%d') for d in date_range[10:13]]

# TODO: write code to create df_iv
df_iv = df.copy()
# Ensure there are columns for customer_value_Y, in_store_visits_T, and received_in_store_re_Z along with various controls like age and income
# df_iv = 

Use `df_iv` to check the compliance between the treatment (`in_store_visits_T`) and randomized encouragement (`received_in_store_re_Z`) i.e. how often people receiving the treatment email actually shopped in store. This is some indication of first stage strength, but we will check it more formally below.

In [ ]:
# TODO: check for compliance (pd.crosstab may be useful)

# The compliance rate should be above 60%

### C. Check the First Stage Strength

Using ols, run a first stage regression to check if the instrument is a strong instrument or not. Is the instrument still valid?

In [ ]:
# TODO: check first stage strength

# Your F Statistic in this first stage regression should be very large i.e. above 100.

### D. Run TSLS to Estimate Our Causal Effect

Now that we have our data and are confident that our instrument is strong and satisfies the necessary assumptions, run two stage least squares with and without control variables.

Note that due to limitations in the statsmodels version of IV regression, we have to manually include an intercept before fitting our model and one hot encode any categorical variables.

Present your results in a stargazer table that cleanly shows the estimated causal effect of in store purchases.

In [ ]:
# TODO: run model without controls

# TODO: run model with controls

In [ ]:
# TODO: use stargazer to display clean output

# If you define the treatment as has an in store transaction, your estimated causal effect should be around 28
# If you define the treatment as count of in store transactions, your estimated causal effect should be around 21
# If you define the treatment as percent of transactions in store, your estimated causal effect should be around 25

What do you conclude from your instrumental variables analysis? How confident are you in the impact of in store shopping on customer value? (Think about any caveats to the IV analysis such as its generalizability.)

## III. Regression Discontinuity
***
<br/>

Concerned that the IV results were on a select part of the user base, you want to run another experiment to assess the value of in store shopping.

After talking to the in store shopping team again, you discover that on October 31, 2020 as part of a Halloween sale, the team provided a coupon to customers living within 15 miles of a store. The coupon was valid till the end of 2021.

Customers living farther than 15 miles unfortunately got no coupon. This was done to save on mailing costs, and the marketing team manager picked 15 cause they are a huge Celtics fan and love Tom Heinsohn (look up "Tommy points" if you are curious).

Fantastic! This sounds like a great discontinuity point for a regression discontinuity design.

If you are having trouble visualizing the problem here, draw a causal diagram that defines the outcome, treatment, and discontinuity variable.

## A. Preparing the Data

Let's create a dataframe called `df_rdd` that limits our sample to the relevant rows. Remember we again need to think carefully about how to aggregate the data to the customer level, how to define our treatment (in store visits), and consider the window we want to use for the RDD.

In this case our discotinuity variable is `distance` with a threshold at 15. How far below and above this discontinuity point should we go for our analysis?

In [ ]:
# time period covered by RDD
discount_date = date_range[9].strftime('%Y-%m-%d')

# TODO: pick a threshold_window above and below the discontinuity point to use
threshold = 15
threshold_window = 0 # Change to appropriate value

df_rdd = df.copy()
df_rdd = df_rdd[(df_rdd['distance'] >= (threshold - threshold_window)) & (df_rdd['distance'] <= (threshold + threshold_window))]

# TODO: Filter transactions to those occuring during the RDD

# TODO: Aggregate dataframe to the customer level and define customer_value_Y, in_store_visits_T, distance_discontinuity, and other controls like age

## A. Check if the Discontinuity is Fuzzy or Sharp

Let's check if the discontinuity point is fuzzy or sharp by checking for compliance with our desired treatment.

In [ ]:
# TODO: check if discontinuity design is fuzzy or sharp (pd.crosstab may be useful)

Since compliance is not perfect, we have a fuzzy discontinuity point. We will need to use `distance_discontinuity` as an instrument and therefore need it to satisfy random assignment, strong first stage, and exclusion restriction.

Ask yourself if you think this instrument is randomly assigned and if it satisfies the exclusion restriction assumption.

## B. Check for Selection and Balance Near the Discontinuity

Check the distribution of some of the controls you included in `df_rdd` on the treated and control populations (based on `distance_discontinuity`).

Do they seem balanced, or is there selection at the threshold on observable variables?

In [ ]:
# TODO: check for covariate balance above and below threshold

Also check for balance in the count of observations near the threshold to see if people have agency on whether they were treated or not. Do the populations above and below seem balanced?

If they are not balanced, do you think this is a sign that people can manipulate where they live in order to impact their chances of getting a coupon?

(Think about if someone would choose where to live in order to get a coupon and if the treatment is therefore impacted by choices people make or not. In causal inference we often can't blindly apply statistical tests and need to consider the context they are operating in too!)

In [ ]:
# TODO: check for observation balance above and below threshold (sns.histplot may be useful)

## C. Graph the Regression Discontinuity

Graph the data using the rdplot function from the rdrobust package. Remember to create a distance variable that is centered at 0 or specify the c argument in the rdplot function.

In [ ]:
# TODO: graph regression discontinuity data

## D. Fit RDD Models for Causal Effects

Now let's fit some regression discontinuity models with and without controls to estimate our causal effect of interest.

Remember because this is a fuzzy design we need to use two stage least squares due to the imperfect compliance with treatment. We can either do that using IV regression or with the fuzzy option in the rdrobust function. In Python it seems that the IV regression route is more stable and has lower variance compared to the rdrobust function, so we suggest using IV2SLS instead.

What do you conclude from your regression discontinuity analysis? How confident are you in the impact of in store shopping on customer value?

In [ ]:
# TODO: fit model without controls

# TODO: fit model with controls

In [ ]:
# TODO: display final results

# If you define the treatment as has an in store transaction, your estimated causal effect should be around 85 for IV2SLS and 185 if you use rdrobust (with a large standard error)
# If you define the treatment as count of in store transactions, your estimated causal effect should be around 13 for IV2SLS and 23 if you use rdrobust (with a large standard error)
# If you define the treatment as percent of transactions in store, your estimated causal effect should be around 56 for IV2SLS and 109 if you use rdrobust (with a large standard error)

## IV. Consolidate the analysis you performed above so it's useful for a stakeholder
***

### Congratulations!

You've done a ton of incredible work. Now, it's time to package it all together so the VP of In Store Sales can follow along.

Again this step often feels like doing an analysis "in reverse." We don't want to step someone through all the logic we just went through to arrive at our answer (as tempting as that might be). We want to share our answer **first,** then help our stakeholders understand it intuitively by sharing visuals and explaining how confident we can be.

Here is a set of suggested steps, but feel free to tweak as you see fit:
- Share the results from your two main modeling approaches, making sure to put the results in **business terms** (e.g., "getting an online customer to shop in store boosts the value of a customer by X; if we can convince 100 online customers to shop in person, we can expect between Y and Z in additional lifetime value")
- Show key visuals to help someone grok the relationships intuitively
- Comment on our degree of confidence of results between the two approaches and whether they are talking about different populations
- Contrast these results with the analysis you presented last week and why it is more or less rigorous in its findings